# Davos Data Project

# Goal:
## Generate data to support a story that demonstrates the value of Gecko's unique dataset


**Hypotheses:**
- Extreme Weather leads to outages due to increased energy demand and stress on the grid
    - Winter weather events , extreme cold
    - Summer and extreme heat
- What is the current state of US boilers, Gecko has inspected many recently, are they in good shape ?  How do we define good shape ?

# Table of Contents:

1. Data Overview
    - data
2. Environment Setup
3. Data Loading & Preprocessing
4. EDA & Visualization

# 1. Data Overview

- data
    - raw_data
        1. **AEP Plants 2011-20220802 Boiler Tube Leaks.xlsx**
            - AEP boiler tube leaks Jan 2011 - Aug 2022
            - Source: AEP, Wolpa
        2. **Duke Fleet Boiler tube failure data 2005 - 2020.csv**
            - Duke boiler tube leaks 2005 - 2020
            - Source: Duke, Connor
        3. **All US Boilers.csv**
            - US Boilers
            - Source: IIR, Wolpa
        - **salesforce.csv**
            - Gecko Salesforce data
            - Source: Gecko Salesforce
        - **portalservice_power_units**
            - all power industry units in portal data
            - Source: Gecko portal-service

# 2. Environment Setup

In [1]:
import json
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import os
import pandas as pd
import requests
import seaborn as sns
import sys
sys.path.insert(0, '..')
import warnings
import zlib

# local
from libs.etl.data_cleaning import generate_aep_outage_dfs, generate_duke_outage_dfs, generate_iir_boilers_dfs, determine_component
from libs.inspection_analysis.utils import analyze_inspection_df, hist_inspection_df, check_thickness, get_thickness_histogram, group_critdat, get_inspection_df
from libs.reference.constants import COMPONENT_TYPE_MAP, POWER_CUST_LIST
from libs.viz.utils import add_vbar_labels, add_hbar_labels

Matplotlib Style:

In [ ]:
plt.style.use('dark_background')
#sns.set_style('darkgrid')

Get Current Working Directory:

In [ ]:
wd = os.getcwd()
print(wd)

Pandas Settings:

In [17]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

Disable Warnings:

In [ ]:
warnings.filterwarnings('ignore')

Set Token ?

In [ ]:
# token = """eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6IkVpeTN0a0l0bzZ0SDFTZ1VZdUdXVCJ9.eyJpc3MiOiJodHRwczovL2dlY2tvcm9ib3RpY3MudXMuYXV0aDAuY29tLyIsInN1YiI6Imdvb2dsZS1vYXV0aDJ8MTAxODIxNjU3MzQ0ODI0ODQxMjA5IiwiYXVkIjpbImNsb3VkLmdlY2tvcm9ib3RpY3MuY29tIiwiaHR0cHM6Ly9nZWNrb3JvYm90aWNzLnVzLmF1dGgwLmNvbS91c2VyaW5mbyJdLCJpYXQiOjE2NzM5NzE1NzYsImV4cCI6MTY3NDA1Nzk3NiwiYXpwIjoiSm1la3lLS0RsM2liRzdQbExRY2JLWHZoekx1UTRaQ08iLCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGVtYWlsIiwicGVybWlzc2lvbnMiOlsiY29sbGVjdGlvbnM6M2RfbW9kZWxzOmVkaXQiLCJjb2xsZWN0aW9uczphZG5vYy1hYnUtZGhhYmktbmF0aW9uYWwtb2lsLWNvbXBhbnktMmE1MzY0OmVkaXQiLCJjb2xsZWN0aW9uczphZXAtYTgyZjE4OmVkaXQiLCJjb2xsZWN0aW9uczphZXMtZWE0ZjMwOmVkaXQiLCJjb2xsZWN0aW9uczpiYXNmLTBmMmNiZTplZGl0IiwiY29sbGVjdGlvbnM6Y2hldnJvbi0wMWE1M2Y6ZWRpdCIsImNvbGxlY3Rpb25zOmNvbWJpbmVkX2NvbXBvbmVudF9kZnM6ZWRpdCIsImNvbGxlY3Rpb25zOmRvbXRhci03NDVhNWE6ZWRpdCIsImNvbGxlY3Rpb25zOmR1a2UtN2YyZmFmOmVkaXQiLCJjb2xsZWN0aW9uczplYmVuc2J1cmctcG93ZXItY29tcGFueS0wZTEyMWY6ZWRpdCIsImNvbGxlY3Rpb25zOmV4eG9uLTdmZDJhYjplZGl0IiwiY29sbGVjdGlvbnM6ZmRlLWRlbW86ZWRpdCIsImNvbGxlY3Rpb25zOmdlb3JnaWEtcGFjaWZpYy1hMWVjNTM6ZWRpdCIsImNvbGxlY3Rpb25zOm1hbnVhbF9pbnNwZWN0aW9uczplZGl0IiwiY29sbGVjdGlvbnM6bm9uc3BlY2lmaWMtY3VzdG9tZXI6ZWRpdCIsImNvbGxlY3Rpb25zOnJvbGxvdXRfaW1hZ2VzOmVkaXQiLCJjb2xsZWN0aW9uczp1cy1uYXZ5LTY5YTcyYTplZGl0IiwiZ2Vja286cmVhZCIsImdlY2tvOnJvbG9kZXg6ZWRpdCJdfQ.k5CJ5-EnyS1ELVIoCtNFqVrYRYuqVsKYdG5_nWqAXxgBcSS-qhiVr-RXpLyX3FLG2BYSrg0qUA8fet27LB7hMUt8wBnFNwFBlHAUbv0xYvyn2zLrxSWMxUIPI5vr6MVRKbJwPpPYVA-uOmM3idW8sWVs3RR544WZUC29Nt73zThAap2LG9qG-UrAJGOo9U365uFKkANH_A69qySfMUnQ7SGrSuuXNP0f8xXPmn4LQ63zJ_kVDEQNNQD-LhQo6ynMVG28h9UhoQ39MPvz-yRJLYKBk45QfQPk1yCqXWTwKKUg4_UyNy-JyaEbTWU3CW_Y47mlqVu5HZHWOWbTE3cb7A"""

# 3. EDA and Visualization

## Source 1:
AEP Plants 2011-20220802 Boiler Tube Leaks.xlsx

Description:

This dataset contains a large set of AEP tube failures across their steam fleet for a number of years

In [ ]:
# get data
aep_df, active_aep_df = generate_aep_outage_dfs()

In [ ]:
aep_df.shape

In [ ]:
active_aep_df.shape

In [ ]:
aep_df.head(1)

In [ ]:
active_aep_df.head(1)

In [ ]:
set(aep_df['System Name'])

In [ ]:
active_aep_df['Event Start Timestamp'].min()

In [ ]:
active_aep_df['Event Start Timestamp'].max()

In [ ]:
active_aep_df['Plant Name'].nunique()

In [ ]:
active_aep_df['Unit Name'].nunique()

In [ ]:
component_df = active_aep_df.groupby(['Cause Description'])['Event Start Timestamp'].nunique().to_frame().sort_values(by='Event Start Timestamp', ascending=False).reset_index()

In [ ]:
fig, ax = plt.subplots()
plt.rcParams["figure.figsize"] = (10,5)

x = component_df["Cause Description"]
y = component_df["Event Start Timestamp"]
width =0.7

rects1 = ax.barh(x, y, width, label = 'Outages')
ax.set_title('AEP Outages by Component')
ax.set_ylabel('Cause')
ax.set_xlabel('Count of Outages')

add_hbar_labels(x, y)

#ax.set_xticks(x)
#ax.set_xticklabels(component_df["Cause Description"])
#ax.legend()

ax.invert_yaxis()

#fig.tight_layout()

#plt.xticks(rotation=90)
plt.show()

In [ ]:
aep_month_outage_df = active_aep_df.groupby(['Outage Month'])['Event Start Timestamp'].nunique().to_frame().reset_index()

In [ ]:
fig, ax = plt.subplots()

x = aep_month_outage_df["Outage Month"]
y = aep_month_outage_df["Event Start Timestamp"]
width =0.7

rects1 = ax.bar(x, y, width, label = 'Outages')
ax.set_title('AEP Outages by Month')
ax.set_ylabel('Count of Outages')
ax.set_xticks(x)
ax.set_xticklabels(aep_month_outage_df["Outage Month"])
#ax.legend()

#add_vbar_labels(x,y)

fig.tight_layout()

plt.xticks(rotation=90)
plt.show()

In [ ]:
plant_outages_df = active_aep_df.groupby(['Plant Name'])['Event Start Timestamp'].nunique().to_frame().reset_index().sort_values(by='Event Start Timestamp', ascending=False)

In [ ]:
fig, ax = plt.subplots()

x = plant_outages_df["Plant Name"]
y = plant_outages_df["Event Start Timestamp"]
width =0.7

rects1 = ax.bar(x, y, width, label = 'Outages')
ax.set_title('Outages by Plant')
ax.set_ylabel('Count of Outages')
ax.set_xticks(x)
ax.set_xticklabels(plant_outages_df["Plant Name"])
ax.legend()

# add_vbar_labels(x,y)

fig.tight_layout()

plt.xticks(rotation=90)
plt.show()

In [ ]:
outage_duration_df = active_aep_df.groupby(['Plant Name'])['NERC Equivalent Hours'].sum().to_frame().reset_index().sort_values(by='NERC Equivalent Hours', ascending=False)

In [ ]:
fig, ax = plt.subplots()

x = outage_duration_df["Plant Name"]
y = outage_duration_df["NERC Equivalent Hours"]
width =0.7

rects1 = ax.bar(x, y, width, label = 'Total Outage Hrs')
ax.set_title('AEP Total Outage Duration by Plant')
ax.set_ylabel('Total Duration (hrs)')
ax.set_xticks(x)
ax.set_xticklabels(plant_outages_df["Plant Name"])
ax.legend()

# add_vbar_labels(x,y)

fig.tight_layout()

plt.xticks(rotation=90)
plt.show()

In [ ]:
outage_mwh_df = active_aep_df.groupby(['Plant Name'])['NERC MWH Loss'].sum().to_frame().reset_index().sort_values(by='NERC MWH Loss', ascending=False)

In [ ]:
fig, ax = plt.subplots()

x = outage_mwh_df["Plant Name"]
y = outage_mwh_df["NERC MWH Loss"]
width =0.7

rects1 = ax.bar(x, y, width, label = 'Total MWhs Lost')
ax.set_title('AEP - Total MWhs Lost by Plant')
ax.set_ylabel('Millions of MWhs')
ax.set_xticks(x)
ax.set_xticklabels(plant_outages_df["Plant Name"])
ax.legend()

#add_vbar_labels(x,y)

fig.tight_layout()

plt.xticks(rotation=90)
plt.show()

## Source 2:
Duke Fleet Boiler tube failure data 2005 - 2020.csv

In [ ]:
duke_df = generate_duke_outage_dfs()

In [ ]:
duke_df.shape

In [ ]:
duke_df.head()

In [ ]:
duke_month_outages_df = duke_df.groupby(['Outage Month'])['Event ID'].nunique().to_frame().reset_index()

In [ ]:
fig, ax = plt.subplots()

x = duke_month_outages_df["Outage Month"]
y = duke_month_outages_df["Event ID"]
width =0.7

rects1 = ax.bar(x, y, width, label = 'Outages')
ax.set_title('Duke Outages by Month')
ax.set_ylabel('Count of Outages')
ax.set_xticks(x)
ax.set_xticklabels(duke_month_outages_df["Outage Month"])
ax.legend()

fig.tight_layout()

plt.xticks(rotation=90)
plt.show()

In [ ]:
duke_station_unit_df = duke_df.groupby(['Station-Unit'])['Event ID'].nunique().to_frame().reset_index().sort_values(by=['Event ID'], ascending=False)

In [ ]:
fig, ax = plt.subplots()

x = duke_station_unit_df["Station-Unit"]
y = duke_station_unit_df["Event ID"]
width =0.7

rects1 = ax.barh(x, y, width, label = 'Outages')
ax.set_title('Duke Outages by Station-Unit')
ax.set_ylabel('Station-Unit')
#ax.set_xticks(x)
#ax.set_xticklabels(duke_station_unit_df["Station-Unit"])
ax.legend()
ax.set_xlabel('Count of Outages')

ax.invert_yaxis()

fig.tight_layout()

#plt.xticks(rotation=90)
plt.show()

## Source 3:
All US Boilers.csv

In [ ]:
us_boilers_df, operational_us_boilers_df = generate_iir_boilers_dfs()

### NOTES:

OUT_DESIGN is in MW

avg 5days per outage

heat rate , lower is better

In [ ]:
us_boilers_df.head(10)

In [ ]:
# us_boilers_df[us_boilers_df['PLANT_NAME'].str.contains('Petersburg')]

In [ ]:
operational_boilers_usage = operational_us_boilers_df.groupby(['PWR_USAGE'])['UNIT_ID'].nunique().to_frame().reset_index()

In [ ]:
fig, ax = plt.subplots()

x = operational_boilers_usage["PWR_USAGE"]
y = operational_boilers_usage["UNIT_ID"]
width =0.7

rects1 = ax.bar(x, y, width, label = 'Status')
ax.set_title('US Boiler Usage Category')
ax.set_ylabel('Count of Units')
ax.set_xticks(x)
#ax.set_xticklabels(x)
#ax.legend()

add_vbar_labels(x,y)

fig.tight_layout()

#plt.axvline(x=2023, c='r')
#plt.xticks(np.arange(min(x), max(x)+1, 5))

#plt.xticks(rotation=45)
plt.show()

In [ ]:
operational_boilers_fuel = operational_us_boilers_df.groupby(['PRIM_FUEL'])['UNIT_ID'].nunique().to_frame().reset_index().sort_values(by=['UNIT_ID'], ascending=False)

In [ ]:
fig, ax = plt.subplots()

x = operational_boilers_fuel["PRIM_FUEL"]
y = operational_boilers_fuel["UNIT_ID"]
width =0.7

rects1 = ax.barh(x, y, width, label = 'Status')
ax.set_title('US Boiler Fuel Type')

#ax.set_xticklabels(x)
ax.legend()

fig.tight_layout()

ax.invert_yaxis()

ax.set_ylabel('Fuel Type')
ax.set_xlabel('Count of Units')
#ax.set_xticks(x)

#plt.axvline(x=2023, c='r')
#plt.xticks(np.arange(min(x), max(x)+1, 5))

#plt.xticks(rotation=90)
plt.show()

In [ ]:
set(us_boilers_df['MARKET_REG'])

In [ ]:
us_boilers_df.groupby(['MARKET_REG', 'PARENTNAME', 'OWNER_NAME', 'OPER_NAME', 'PLANT_NAME'])['UNIT_ID'].nunique().to_frame().head(20)

In [ ]:
us_boilers_df['PARENTNAME'].nunique()

In [ ]:
us_boilers_df['UNIT_ID'].nunique()

In [ ]:
shutdowns_by_yr = us_boilers_df.groupby(['SHUTDOWN_yr'])['UNIT_ID'].nunique().to_frame().reset_index()

In [ ]:
shutdowns_by_yr.head()

In [ ]:
shutdowns_by_yr[shutdowns_by_yr.UNIT_ID==shutdowns_by_yr.UNIT_ID.max()]

In [ ]:
fig, ax = plt.subplots()

x = shutdowns_by_yr["SHUTDOWN_yr"]
y = shutdowns_by_yr["UNIT_ID"]
width =0.7

rects1 = ax.bar(x, y, width, label = 'Planned Shutdowns')
ax.set_title('US Boiler Shutdowns')
ax.set_ylabel('Count of Units')
ax.set_xticks(x)
#ax.set_xticklabels(shutdowns_by_yr["SHUTDOWN_yr"])
ax.legend()

fig.tight_layout()

plt.axvline(x=2023, c='r')
plt.xticks(np.arange(min(x), max(x)+1, 5))

plt.xticks(rotation=45)
plt.show()

In [ ]:
operational_boilers_shutdown = operational_us_boilers_df.groupby(['SHUTDOWN_yr'])['UNIT_ID'].nunique().to_frame().reset_index()

In [ ]:
fig, ax = plt.subplots()

x = operational_boilers_shutdown["SHUTDOWN_yr"]
y = operational_boilers_shutdown["UNIT_ID"]
width =0.7

rects1 = ax.bar(x, y, width, label = 'Planned Shutdowns')
ax.set_title('US Boiler Shutdowns')
ax.set_ylabel('Count of Units')
ax.set_xticks(x)
#ax.set_xticklabels(shutdowns_by_yr["SHUTDOWN_yr"])
ax.legend()

fig.tight_layout()

plt.axvline(x=2023, c='r')
plt.xticks(np.arange(min(x), max(x)+1, 5))

plt.xticks(rotation=45)
plt.show()

In [ ]:
boiler_status = us_boilers_df.groupby(['U_STATUS'])['UNIT_ID'].nunique().to_frame().reset_index()

In [ ]:
fig, ax = plt.subplots()

x = boiler_status["U_STATUS"]
y = boiler_status["UNIT_ID"]
width =0.7

rects1 = ax.bar(x, y, width, label = 'Status')
ax.set_title('US Boiler Status')
ax.set_ylabel('Count of Units')
ax.set_xticks(x)
#ax.set_xticklabels(x)
ax.legend()

fig.tight_layout()

#plt.axvline(x=2023, c='r')
#plt.xticks(np.arange(min(x), max(x)+1, 5))

plt.xticks(rotation=45)
plt.show()

Mothballed power plants
Power plant layup or mothballing is one of the methods that plant operators can apply to prevent corrosive damage and ensure the preservation of various assets. When equipment and plants are mothballed, the organization maintains the service life of these facilities by employing various protective measures.

https://www.bryair.com/news-and-events/articles/mothballing-as-an-effective-power-plant-layup/#:~:text=Power%20plant%20layup%20or%20mothballing,by%20employing%20various%20protective%20measures.

In [ ]:
us_boilers_byparent = operational_us_boilers_df.groupby(['PARENTNAME'])['UNIT_ID'].nunique().to_frame().sort_values(by='UNIT_ID', ascending=False).reset_index()

In [ ]:
us_boilers_byparent.head(10)

In [ ]:
us_boilers_byparent.UNIT_ID.sum()

In [ ]:
us_boilers_df.groupby(['FUEL_GROUP', 'PRIM_FUEL', 'SECND_FUEL'])['UNIT_ID'].nunique().to_frame().sort_values(by='UNIT_ID', ascending=False).head(20)

In [ ]:
operational_us_boilers_df.groupby(['FUEL_GROUP'])['UNIT_ID'].nunique().to_frame().sort_values(by='UNIT_ID', ascending=False)

### Source 4:

Portal Service ...


portalservice_power_units.json

In [3]:
with open(r'../data/raw_data/portalservice_power_inspections_2022.json') as json_data:
    d = json.loads(json_data.read())
    json_data.close()

In [4]:
portalservice_power_inspections_2022_df = pd.json_normalize(d)

In [ ]:
portalservice_power_inspections_2022_df.head(1)

In [ ]:
portalservice_power_inspections_2022_df.columns

In [ ]:
ref_df = portalservice_power_inspections_2022_df.loc[:,(
    'Customer.name',
    'Location.region',
    'Location.name',
    'Location.city',
    'Location.slack_channel',
    'Unit.slug',
    'Unit.id',
    'Unit.name',
    'Unit.retirement_date',
    'Unit.fuel_type',
    'Unit.capacity',
    'Unit.total_tubes',
    'Component.slug',
    'Component.name',
    'Component.tube_diameter', 
    'Component.tube_spacing',
    'Component.shell_diameter',
    'Component.component_type',
    'Inspection.slug',
    'Inspection.date',
    'Inspection.ndt_method_type',
    'Inspection.inspection_type',
)]

In [ ]:
ref_df.head(1)

In [ ]:
ref_df.shape

In [ ]:
min(ref_df['Inspection.date'])

In [ ]:
max(ref_df['Inspection.date'])

In [ ]:
set(ref_df['Component.component_type'])

In [ ]:
ref_df[ref_df['Customer.name']=='Consolidated Edison Company of New York (Corporate)']

In [ ]:
portalservice_power_inspections_2022_df.shape

# For Wolpa



In [2]:
wolpas_df = pd.read_csv(r'../data/raw_data/unit_slug_mappings.csv')

In [ ]:
wolpas_df.head()

In [ ]:
wolpas_df.shape

In [ ]:
wolpas_df[~wolpas_df.GECKO_UNIT_SLUG.isnull()].shape

In [ ]:
filtered_wolpa_df = wolpas_df[~wolpas_df['GECKO_UNIT_SLUG'].isnull()]

In [ ]:
filtered_wolpa_df.shape

In [ ]:
filtered_wolpa_df.head()

In [ ]:
slug_maps = ref_df.loc[:,('Inspection.slug', 'Unit.slug')]

In [ ]:
wolpa_df_inspection_slugs = filtered_wolpa_df.merge(slug_maps, how='left', left_on='GECKO_UNIT_SLUG', right_on='Unit.slug')

In [ ]:
wolpa_df_inspection_slugs.head()

## Get All PBI

In [ ]:
all_pbi_to_check_df = pd.read_csv('../data/outputs/all_pbi.csv')

In [ ]:
all_pbi_to_check_df.head(1)

In [ ]:
all_pbi_to_check_df.shape

## Get Slopes Components Only

In [ ]:
all_pbi_to_check_df['Component_type'] = all_pbi_to_check_df['Component.slug'].apply(lambda x: determine_component(x))

In [ ]:
set(all_pbi_to_check_df['Component_type'])

In [ ]:
all_pbi_to_check_df.groupby(['Component_type'])['Component.slug'].nunique().to_frame()

In [ ]:
slope_slugs_df = all_pbi_to_check_df[all_pbi_to_check_df['Component_type']=='Slopes']

In [ ]:
slope_slugs_df.head()

In [ ]:
slope_slugs_df.groupby(['Customer.name', 'Location.name'])['Component.slug'].nunique()

In [ ]:
slope_slugs_df.rename(columns={
    'Inspection.slug': 'slug',	
    'Component.wall_nominal_thickness': 'nominal',
}, inplace=True)

In [ ]:
slope_slugs_df['nominal'].isnull().sum()

In [ ]:
slope_slugs_df.shape[0]

In [ ]:
slope_slugs_df[slope_slugs_df['nominal']==0.012]

In [ ]:
slopes_df = slope_slugs_df[(~slope_slugs_df.nominal.isnull()) & (slope_slugs_df.nominal!=0.012)]

In [ ]:
set(slopes_df['nominal'])

In [ ]:
slopes_df.shape

In [ ]:
slopes_df[slopes_df['nominal']<0.22].shape

In [ ]:
slopes_df.reset_index(inplace=True)

In [ ]:
slopes_histogram_df, slopes_hist_error_slug_list = get_thickness_histogram(slopes_df)

In [ ]:
slopes_comb_df = slopes_df.merge(slopes_histogram_df, how='inner', on=['slug', 'nominal'])

In [ ]:
slopes_comb_df.drop(columns='index', inplace=True)

In [ ]:
slopes_comb_df.head()

In [ ]:
slopes_comb_df.shape

In [ ]:
# slopes_comb_df.to_csv('../data/outputs/raw_slopes.csv')

In [ ]:
slopes_comb_df[(slopes_comb_df['Customer.name'].isin(POWER_CUST_LIST))].shape

In [ ]:
slopes_comb_df[(slopes_comb_df['Customer.name'].isin(POWER_CUST_LIST)) & (slopes_df['nominal']<0.24)].shape

In [ ]:
slopes_comb_df[(slopes_comb_df['Customer.name'].isin(POWER_CUST_LIST)) & (slopes_df['nominal']<0.22)].shape

In [ ]:
slopes_comb_df.columns

In [ ]:
slopes_comb_df.groupby(['Component_type'])[
    'Bins w 10% Loss',
    'Bins w 20% Loss',
    'Bins w 30% Loss',
    'Bins w 40% Loss',
    'Bins w 50% Loss',
    'Bins w 60% Loss',
    'Bins w 70% Loss',
    'Bins w 80% Loss',
    'Bins w 90% Loss',
    'nominal',
    'tubes_inspected',
    'bins_collected'
].median()

In [ ]:
slopes_comb_df[(slopes_comb_df['Customer.name'].isin(POWER_CUST_LIST))].groupby(['Component_type'])[
    'Bins w 10% Loss',
    'Bins w 20% Loss',
    'Bins w 30% Loss',
    'Bins w 40% Loss',
    'Bins w 50% Loss',
    'Bins w 60% Loss',
    'Bins w 70% Loss',
    'Bins w 80% Loss',
    'Bins w 90% Loss',
    'nominal',
    'tubes_inspected',
    'bins_collected'
].median()

In [ ]:
slopes_comb_df[(slopes_comb_df['Customer.name'].isin(POWER_CUST_LIST)) & (slopes_df['nominal']<0.22)].groupby(['Component_type'])[
    'Bins w 10% Loss',
    'Bins w 20% Loss',
    'Bins w 30% Loss',
    'Bins w 40% Loss',
    'Bins w 50% Loss',
    'Bins w 60% Loss',
    'Bins w 70% Loss',
    'Bins w 80% Loss',
    'Bins w 90% Loss',
    'nominal',
    'tubes_inspected',
    'bins_collected'
].median()

In [ ]:
slopes_comb_df[(slopes_comb_df['Customer.name'].isin(POWER_CUST_LIST)) & (slopes_df['nominal']>=0.24)].shape

In [ ]:
slopes_comb_df[(slopes_comb_df['Customer.name'].isin(POWER_CUST_LIST)) & (slopes_df['nominal']>0.24)].groupby(['Component_type'])[
    'Bins w 10% Loss',
    'Bins w 20% Loss',
    'Bins w 30% Loss',
    'Bins w 40% Loss',
    'Bins w 50% Loss',
    'Bins w 60% Loss',
    'Bins w 70% Loss',
    'Bins w 80% Loss',
    'Bins w 90% Loss',
    'nominal',
    'tubes_inspected',
    'bins_collected'
].median()

In [ ]:
slopes_comb_df[(slopes_comb_df['Customer.name'].isin(POWER_CUST_LIST)) & (slopes_df['nominal']<=0.24)].shape

In [ ]:
slopes_comb_df[(slopes_comb_df['Customer.name'].isin(POWER_CUST_LIST)) & (slopes_df['nominal']<=0.24)].groupby(['Component_type'])[
    'Bins w 10% Loss',
    'Bins w 20% Loss',
    'Bins w 30% Loss',
    'Bins w 40% Loss',
    'Bins w 50% Loss',
    'Bins w 60% Loss',
    'Bins w 70% Loss',
    'Bins w 80% Loss',
    'Bins w 90% Loss',
    'nominal',
    'tubes_inspected',
    'bins_collected'
].median()

In [ ]:
slopes_comb_df[slopes_comb_df['Customer.name']=='Duke Energy']

In [ ]:
IIR_ID_slopes_df = pd.read_csv('../data/intermediate/slopes_wIIR_id.csv')

In [ ]:
IIR_ID_slopes_df.head(1)

In [ ]:
IIR_ID_slopes_df.IIR_UNIT_ID = IIR_ID_slopes_df.IIR_UNIT_ID.astype(int)

In [ ]:
us_boilers_df.head(1)

In [ ]:
enriched_slopes_aggs_df = IIR_ID_slopes_df.merge(us_boilers_df, how='left', left_on='IIR_UNIT_ID', right_on='UNIT_ID')

In [ ]:
enriched_slopes_aggs_df.head(1)

In [ ]:
enriched_slopes_aggs_df[(enriched_slopes_aggs_df['Customer.name'].isin(POWER_CUST_LIST)) & (enriched_slopes_aggs_df['nominal']<=0.24)].groupby(['FUEL_GROUP'])['slug'].nunique()

In [ ]:
enriched_slopes_aggs_df[(enriched_slopes_aggs_df['Customer.name'].isin(POWER_CUST_LIST)) & (enriched_slopes_aggs_df['nominal']<=0.24)].groupby(['PRIM_FUEL'])['slug'].nunique()

In [ ]:
enriched_slopes_aggs_df.shape

In [ ]:
enriched_slopes_aggs_df.to_csv('../data/outputs/agg_slopes_data.csv')

In [ ]:
all_pbi_to_check_df.rename(columns={
    'Inspection.slug': 'slug',	
    'Component.wall_nominal_thickness': 'nominal',
}, inplace=True)

null_nominals = all_pbi_to_check_df['nominal'].isnull().sum()
null_nominals_aspercentage = round(100*null_nominals/all_pbi_to_check_df.shape[0], 2)

median_nominal = all_pbi_to_check_df['nominal'].median()

print('Nominals with null value: {} , or {}% of total'.format(null_nominals, null_nominals_aspercentage))
print('Nominal used to fillna:  {}'.format(median_nominal))

all_pbi_to_check_df['nominal'].fillna(median_nominal, inplace=True)

## Error checking and handling

In [ ]:
len(error_slug_list)

In [ ]:
error_slug_list

In [ ]:
error_df = ref_df[ref_df['Inspection.slug'].isin(error_slug_list)]

In [ ]:
dominion_mtstorm_unit_1 = ['20220919-072202', '20220919-f40fdc', '20220919-35548d', '20220917-b520a5']
great_river_spiritwood_unit_1 = ['20220507-caec3a', '20220507-182a26', '20220506-c90d1d', '20220506-396ef9']
aes_warrior_run_unit_1 = ['20220509-27e470', '20220509-addaf6', '20220509-0849bd']

In [ ]:
unchecked_error_df = error_df[
    (~error_df['Inspection.slug'].isin(dominion_mtstorm_unit_1)) & 
    (~error_df['Inspection.slug'].isin(great_river_spiritwood_unit_1)) &
    (~error_df['Inspection.slug'].isin(aes_warrior_run_unit_1))
]

In [ ]:
unchecked_error_df

In [ ]:
#get_inspection_df('20220310-990386')

## Davos - Jake

In [ ]:
jake_davos_threshold = 0.6
# jake_davos_df, davos_error_slug_list = check_thickness(all_pbi_to_check_df, jake_davos_threshold)

In [ ]:
# ratio = 100*(jake_davos_df[jake_davos_df.critical_bins != 0].shape[0] / jake_davos_df.shape[0])

In [ ]:
# ratio

In [ ]:
round(ratio, 2)

In [ ]:
len(davos_error_slug_list)

# Use Histogram Generator

In [ ]:
# histogram_df, hist_error_slug_list = get_thickness_histogram(all_pbi_to_check_df)

In [ ]:
histogram_df.head()

In [ ]:
histogram_df.shape

In [ ]:
len(hist_error_slug_list)

In [ ]:
enriched_hist_df = histogram_df.merge(all_pbi_to_check_df.loc[:,(
    'slug',
    'Customer.name',
    'Location.name',
    'Location.city',
    'Unit.name',
    'Unit.slug',
    'Component.slug',
)], how='left', on='slug')

In [ ]:
enriched_hist_df.head(1)

In [ ]:
enriched_hist_df.shape

In [ ]:
clean_hist_df = enriched_hist_df.loc[:, (
    'Customer.name',
    'Location.name',
    'Location.city',
    'Unit.name',
    'Unit.slug',
    'Component.slug',
    'slug',
    'nominal',
    'min_t',
    'max_t',
    'tubes_inspected',
    'bins_collected',
    'Tubes w 10% Loss',
    'Tubes w 20% Loss',
    'Tubes w 30% Loss',
    'Tubes w 40% Loss',
    'Tubes w 50% Loss',
    'Tubes w 60% Loss',
    'Tubes w 70% Loss',
    'Tubes w 80% Loss',
    'Tubes w 90% Loss',
    'Bins w 10% Loss',
    'Bins w 20% Loss',
    'Bins w 30% Loss',
    'Bins w 40% Loss',
    'Bins w 50% Loss',
    'Bins w 60% Loss',
    'Bins w 70% Loss',
    'Bins w 80% Loss',
    'Bins w 90% Loss'
)]

In [ ]:
# NOTE this relies on an assume 3in bin size
clean_hist_df['tube_feet_inspected'] = clean_hist_df['bins_collected']*3/12 # convert to tube feet ?

In [ ]:
clean_hist_df.head(1)

In [ ]:
clean_hist_df.shape

## Filter for just the power customers identified

In [ ]:
power_hist_df = clean_hist_df[clean_hist_df['Customer.name'].isin(POWER_CUST_LIST)]

In [ ]:
site_grouped_hist = clean_hist_df.groupby(['Customer.name', 'Location.name'])[
    'Tubes w 10% Loss',
    'Tubes w 20% Loss',
    'Tubes w 30% Loss',
    'Tubes w 40% Loss',
    'Tubes w 50% Loss',
    'Tubes w 60% Loss',
    'Tubes w 70% Loss',
    'Tubes w 80% Loss',
    'Tubes w 90% Loss',
    'Bins w 10% Loss',
    'Bins w 20% Loss',
    'Bins w 30% Loss',
    'Bins w 40% Loss',
    'Bins w 50% Loss',
    'Bins w 60% Loss',
    'Bins w 70% Loss',
    'Bins w 80% Loss',
    'Bins w 90% Loss',
    'tube_feet_inspected',
    'tubes_inspected',	
    'bins_collected'
].sum()

In [ ]:
site_grouped_hist.head(25)

In [ ]:
unit_grouped_hist = clean_hist_df.groupby(['Customer.name', 'Location.name', 'Unit.name'])[
    'Tubes w 10% Loss',
    'Tubes w 20% Loss',
    'Tubes w 30% Loss',
    'Tubes w 40% Loss',
    'Tubes w 50% Loss',
    'Tubes w 60% Loss',
    'Tubes w 70% Loss',
    'Tubes w 80% Loss',
    'Tubes w 90% Loss',
    'Bins w 10% Loss',
    'Bins w 20% Loss',
    'Bins w 30% Loss',
    'Bins w 40% Loss',
    'Bins w 50% Loss',
    'Bins w 60% Loss',
    'Bins w 70% Loss',
    'Bins w 80% Loss',
    'Bins w 90% Loss',
    'tube_feet_inspected',
    'tubes_inspected',	
    'bins_collected'
].sum()

In [ ]:
unit_grouped_hist.head()

In [ ]:
unit_grouped_hist[unit_grouped_hist['Bins w 40% Loss']!=0].shape

In [ ]:
unit_grouped_hist[unit_grouped_hist['Bins w 50% Loss']!=0].shape

In [ ]:
unit_grouped_hist[unit_grouped_hist['Bins w 60% Loss']!=0].shape

In [ ]:
unit_grouped_hist.shape

Save unit_grouped_hist as a csv

In [ ]:
# unit_grouped_hist.reset_index().to_csv('data/outputs/unit_grouped_hist.csv')

In [ ]:
component_grouped_hist = clean_hist_df.groupby(['Customer.name', 'Location.name', 'Unit.name', 'Component.slug'])[
    'Tubes w 10% Loss',
    'Tubes w 20% Loss',
    'Tubes w 30% Loss',
    'Tubes w 40% Loss',
    'Tubes w 50% Loss',
    'Tubes w 60% Loss',
    'Tubes w 70% Loss',
    'Tubes w 80% Loss',
    'Tubes w 90% Loss',
    'Bins w 10% Loss',
    'Bins w 20% Loss',
    'Bins w 30% Loss',
    'Bins w 40% Loss',
    'Bins w 50% Loss',
    'Bins w 60% Loss',
    'Bins w 70% Loss',
    'Bins w 80% Loss',
    'Bins w 90% Loss',
    'tube_feet_inspected',
    'tubes_inspected',	
    'bins_collected'
].sum()

In [ ]:
component_grouped_hist.tail(5)

In [ ]:
component_grouped_hist.shape

My Thoughts:

- we probably want to find a better way to combine aggregates for the same component slug, we currently sum, but if those are 2 separate inspections did they cover the same section ?  Were they at different dates ?  there is more nuance that is not being addressed here

- how could we identify weld overlay ?
- how can we incorporate the partial inspections into a better model

David Wolpa
 
Here are my ideas to increase the value of this data asset:

1. Group the slugs into waterwalls and slopes
2. include partial boiler scans (vs just relying on full scans) by grouping data into critical bins per 1000 (or some other number) inspected. This makes our sample size much larger improving accuracy and ability to slice data…
3. Add meta data like boiler type, and weld overlay amount that we can later filter on to produce most accurate comparisons groups
4. bring into radar like y'all are discussing to hydrate initial predictions before we even step foot into a specific boiler
5. The sales team already has a tool that calculates how many tube feet a specific boiler has based on eng drawings so we can use that to normalize what we expect to find in specific boiler to our data set that is orgianized by critical bins per 1000 bins inspected

## Bowen Analysis

In [ ]:
bowen_df = clean_hist_df[
    (clean_hist_df['Customer.name'].str.contains('Southern Co')) & 
    (clean_hist_df['Location.name'].str.contains('Plant Bowen'))
]

In [ ]:
bowen_df.head(1)

In [ ]:
bowen_df.groupby(['Unit.name'])['slug'].nunique().to_frame()

In [ ]:
bowen_unit_1_df = bowen_df[bowen_df['Unit.name']=='Unit 1']

In [ ]:
bowen_unit_1_df

In [ ]:
# bowen_unit_1_df.to_csv('../data/outputs/bowen.csv')

In [ ]:
hist_values_sum = unit_grouped_hist.sum()
hist_cols = unit_grouped_hist.columns

In [ ]:
hist_values_median = unit_grouped_hist.median()

In [ ]:
hist_values_mean = unit_grouped_hist.mean()

In [ ]:
hist_cols

In [ ]:
fig, ax = plt.subplots()

x = hist_cols[9:18]
y = hist_values_sum[9:18]
width =0.7

rects1 = ax.barh(x, y, width, label = 'Status')
ax.set_title('US Boiler Wall Loss Histogram')
ax.set_ylabel('Count of Bins')
#ax.set_xticks(x)
#ax.set_xticklabels(x)
ax.legend()

fig.tight_layout()

#plt.axvline(x=2023, c='r')
#plt.xticks(np.arange(min(x), max(x)+1, 5))

plt.xticks(rotation=0)
plt.show()

In [ ]:
fig, ax = plt.subplots()

x = hist_cols[0:9]
y = hist_values_sum[9:18]
width =0.7

rects1 = ax.barh(x, y, width, label = 'Status')
ax.set_title('US Boiler Wall Loss Histogram')
ax.set_ylabel('Count of Tubes')
#ax.set_xticks(x)
#ax.set_xticklabels(x)
ax.legend()

fig.tight_layout()

#plt.axvline(x=2023, c='r')
#plt.xticks(np.arange(min(x), max(x)+1, 5))

plt.xticks(rotation=0)
plt.show()

In [ ]:
fig, ax = plt.subplots()

x = hist_cols[9:18]
y = hist_values_median[9:18]
width =0.7

rects1 = ax.barh(x, y, width, label = 'Status')
ax.set_title('US Boiler Wall Loss Histogram')
ax.set_ylabel('Count of Bins')
#ax.set_xticks(x)
#ax.set_xticklabels(x)
ax.legend()

fig.tight_layout()

#plt.axvline(x=2023, c='r')
#plt.xticks(np.arange(min(x), max(x)+1, 5))

plt.xticks(rotation=0)
plt.show()

In [ ]:
fig, ax = plt.subplots()

x = hist_cols[0:9]
y = hist_values_median[0:9]
width =0.7

rects1 = ax.barh(x, y, width, label = 'Status')
ax.set_title('US Boiler Wall Loss Histogram')
ax.set_ylabel('Count of Bins')
#ax.set_xticks(x)
#ax.set_xticklabels(x)
ax.legend()

fig.tight_layout()

#plt.axvline(x=2023, c='r')
#plt.xticks(np.arange(min(x), max(x)+1, 5))

plt.xticks(rotation=0)
plt.show()

In [ ]:
fig, ax = plt.subplots()

x = hist_cols[9:18]
y = hist_values_mean[9:18]
width =0.7

rects1 = ax.barh(x, y, width, label = 'Status')
ax.set_title('US Boiler Wall Loss Histogram')
ax.set_ylabel('Count of Bins')
#ax.set_xticks(x)
#ax.set_xticklabels(x)
ax.legend()

fig.tight_layout()

#plt.axvline(x=2023, c='r')
#plt.xticks(np.arange(min(x), max(x)+1, 5))

plt.xticks(rotation=0)
plt.show()

In [ ]:
fig, ax = plt.subplots()

x = hist_cols[0:9]
y = hist_values_mean[0:9]
width =0.7

rects1 = ax.barh(x, y, width, label = 'Status')
ax.set_title('US Boiler Wall Loss Histogram')
ax.set_ylabel('Count of Bins')
#ax.set_xticks(x)
#ax.set_xticklabels(x)
ax.legend()

fig.tight_layout()

#plt.axvline(x=2023, c='r')
#plt.xticks(np.arange(min(x), max(x)+1, 5))

plt.xticks(rotation=0)
plt.show()

In [ ]:
unit_grouped_hist.head(1)

In [ ]:
component_grouped_hist.head(1)

In [ ]:
ref_df.head(1)

## Merge crit df and ref df into comb_df

In [ ]:
comb_df = unit_grouped_hist.merge(ref_df, how='left', on=[
    'Customer.name', 
    'Location.name',
    'Unit.name'
])

In [ ]:
comb_df.head()

In [ ]:
component_comb_df = component_grouped_hist.merge(ref_df, how='left', on=[
    'Customer.name', 
    'Location.name',
    'Unit.name',
    'Component.slug'
])

In [ ]:
component_comb_df.tail()

In [ ]:
def determine_component(string):
    """
    Take a component slug and derive the component type
    """
    component_type = 'Other'
    
    if 'slopes' in str(string).lower():
        component_type = 'Slopes'
    if 'walls' in str(string).lower():
        component_type = 'Walls'
    
    return component_type

In [ ]:
component_comb_df['Component_type'] = component_comb_df['Component.name'].apply(lambda x: determine_component(x))

Ideas here:

- if string component.name contains wall
- if string component name contains slope

In [ ]:
set(component_comb_df['Component.name'])

In [ ]:
set(component_comb_df['Component_type'])

In [ ]:
component_comb_df.groupby(['Component_type'])['Component.slug'].nunique().to_frame()

In [ ]:
with open(r'../data/raw_data/power_boiler_inspections_pre2019.json') as json_data:
    d = json.loads(json_data.read())
    json_data.close()

In [ ]:
power_boiler_inspections19_df = pd.json_normalize(d)

In [ ]:
power_boiler_inspections19_df['Unit.id'].nunique()

In [ ]:
power_boiler_inspections19_df.head(1)

In [ ]:
min(power_boiler_inspections19_df['Inspection.date'])

In [ ]:
max(power_boiler_inspections19_df['Inspection.date'])

In [ ]:
filtered_pre19_power_boiler_inspections_df = power_boiler_inspections19_df.loc[:, (
    'Inspection.slug', 
    'Component.wall_nominal_thickness', 
    'Component.wall_warning_thickness', 
    'Component.coating_nominal_thickness',
    'Component.slug',
    'Unit.name',
    'Unit.slug',
    'Location.name',
    'Location.city',
    'Customer.name'
)]

In [ ]:
with open(r'../data/raw_data/power_boiler_inspections.json') as json_data:
    d = json.loads(json_data.read())
    json_data.close()

In [ ]:
power_boiler_inspections_df = pd.json_normalize(d)

In [ ]:
power_boiler_inspections_df.head(1)

In [ ]:
filtered_power_boiler_inspections_df = power_boiler_inspections_df.loc[:, (
    'Inspection.slug', 
    'Component.wall_nominal_thickness', 
    'Component.wall_warning_thickness', 
    'Component.coating_nominal_thickness',
    'Component.slug',
    'Unit.name',
    'Unit.slug',
    'Location.name',
    'Location.city',
    'Customer.name'
)]

In [ ]:
all_pbi_df = pd.concat([filtered_power_boiler_inspections_df, filtered_pre19_power_boiler_inspections_df])

In [ ]:
all_pbi_df.head(1)

In [ ]:
all_pbi_ordered_df = all_pbi_df.loc[:,(
    'Customer.name', 
    'Location.name', 
    'Location.city',
    'Location.region',
    'Location.slack_channel',
    'Unit.id',
    'Unit.name', 
    'Unit.slug',
    'Unit.retirement_date',
    'Unit.fuel_type',
    'Unit.capacity',
    'Unit.total_tubes',
    'Component.slug', 
    'Component.component_type',
    'Component.wall_nominal_thickness', 
    'Component.wall_warning_thickness', 
    'Component.coating_nominal_thickness',
    'Component.tube_diameter', 
    'Component.tube_spacing',
    'Component.shell_diameter',
    'Inspection.slug',
    'Inspection.date',
    'Inspection.ndt_method_type',
    'Inspection.inspection_type'
)].drop_duplicates()

In [ ]:
all_pbi_ordered_df.head(1)

In [ ]:
all_pbi_ordered_df.shape

In [ ]:
# all_pbi_ordered_df.to_csv('../data/outputs/all_pbi.csv')

In [ ]:
ref_df = portalservice_power_inspections_2022_df.loc[:,(

    'Inspection.date',
    'Inspection.ndt_method_type',
    'Inspection.inspection_type',

    'Component.tube_diameter', 
    'Component.tube_spacing',
    'Component.shell_diameter',
    'Component.component_type',

)]

In [ ]:
names_df = all_pbi_ordered_df.loc[:, (
    'Customer.name', 
    'Location.name', 
    'Location.city', 
    'Unit.name', 
    'Unit.slug'
)].drop_duplicates()

In [ ]:
names_df.shape

In [ ]:
power_parents = [
    'AEP',
    'AES',
    'ALCOA',
    'Dominion',
    'Duke Energy',
    'East Kentucky Power Coop (EKPC)',
    'Hallador Power Company',
    'NRG',
    'Southern Co'
]

In [ ]:
power_names_df = names_df[names_df['Customer.name'].isin(power_parents)]

In [ ]:
power_names_df.shape

In [ ]:
power_names_df.sort_values(['Customer.name', 'Location.name', 'Location.city', 'Unit.name'])

In [ ]:
power_boiler_inspections_df['Unit.id'].nunique()

In [ ]:
power_boiler_inspections_df['Inspection.id'].nunique()

In [ ]:
power_boiler_inspections_df.groupby([])

In [ ]:
aep_month_outage_df.head()

In [ ]:
duke_month_outages_df.head()

In [ ]:
aep_month_outage_df.rename(columns={'Event Start Timestamp': 'Event Count'}, inplace=True)
duke_month_outages_df.rename(columns={'Event ID': 'Event Count'}, inplace=True)
aep_month_outage_df['Utility'] = 'AEP'
duke_month_outages_df['Utility'] = 'Duke'

In [ ]:
eia_df = pd.read_csv(r'../data/raw_data/EIA.csv')

In [ ]:
eia_df.head()

In [ ]:
eia_df.shape

In [ ]:
eia_df['Year'] = eia_df['YYYYMM'].astype(str).str[0:4]
eia_df['Month'] = eia_df['YYYYMM'].astype(str).str[4:6]

In [ ]:
set(eia_df['Month'])

In [ ]:
recent_eia_df = eia_df[eia_df['Year'].astype(int) > 2000]

In [ ]:
recent_eia_df.shape

In [ ]:
set(recent_eia_df['Month'])

In [ ]:
set(recent_eia_df["Description"])

In [ ]:
keep_cols = ['Total Primary Energy Consumption',
 'Total Primary Energy Production']

In [ ]:
eia_prim_en_df = recent_eia_df[recent_eia_df["Description"].isin(keep_cols)]

In [ ]:
eia_prim_en_df.shape

In [ ]:
eia_prim_en_df.head()

In [ ]:
eia_2021_df = eia_prim_en_df[(eia_prim_en_df['Year']=='2021') & (eia_prim_en_df['Month']!='13')]

In [ ]:
eia_2021_df.head()

In [ ]:
eia_2021_consumption = eia_2021_df[eia_2021_df['Description']=='Total Primary Energy Consumption']
eia_2021_production = eia_2021_df[eia_2021_df['Description']=='Total Primary Energy Production']

In [ ]:
fig=plt.figure()
fig.show()
ax=fig.add_subplot(111)

ax.set_title('EIA US Energy Data')
ax.set_ylabel('Quadrillion Btu')
ax.set_ylim(eia_2021_consumption['Value'].min()-1,10)
ax.set_xlabel('Month')
ax.set_xticks([1,2,3,4,5,6,7,8,9,10,11,12])
ax.set_xticklabels(eia_2021_consumption['Month'])


ax.plot(eia_2021_consumption['Month'].astype(int), eia_2021_consumption['Value'], c='r', label='Primary Energy Consumption')
#ax.plot(eia_2021_production['Month'].astype(int), eia_2021_production['Value'], c='g', label='Primary Energy Production')


plt.legend()
plt.draw()

In [ ]:
fig=plt.figure()
fig.show()
ax=fig.add_subplot(111)

ax.set_title('Tube Failures by Month')
ax.set_ylabel('Count of Tube Failures')
ax.set_xlabel('Month')
ax.set_xticks(aep_month_outage_df['Outage Month'])
ax.set_xticklabels(aep_month_outage_df['Outage Month'])


ax.plot(aep_month_outage_df['Outage Month'], aep_month_outage_df['Event Count'], c='r', label='AEP Tube Failures')
ax.plot(duke_month_outages_df['Outage Month'], duke_month_outages_df['Event Count'], c='b', label='Duke Tube Failures')


plt.legend()
plt.draw()

In [ ]:
comb_df.head()

In [ ]:
comb_df['Inspection.month'] = pd.to_datetime(comb_df['Inspection.date']).dt.month

In [ ]:
#set(comb_df['Inspection.month'])

In [ ]:
gecko_2022_inspections = comb_df.groupby(['Inspection.month'])['Inspection.slug'].nunique().to_frame().reset_index().rename(columns={'slug': 'inspections'})

In [ ]:
gecko_2022_inspections

In [ ]:
gecko_2022_inspections['Inspection.slug'].sum()

In [ ]:
#us_boilers_df.loc[:, ('MARKET_REG', 'PLANT_NAME')]

In [ ]:
us_boilers_df.loc[:, ('MARKET_REG', 'UNIT_STATE')]

In [ ]:
len(comb_df['Location.city'])

In [ ]:
len([city for city in comb_df['Location.city'] if city not in us_boilers_df['PHYS_CITY']])

In [ ]:
set(us_boilers_df['MARKET_REG'])

In [ ]:
markets_comb_df = comb_df.merge(us_boilers_df.loc[:, ('MARKET_REG', 'UNIT_STATE')], how='left', left_on='Location.region', right_on='UNIT_STATE')

In [ ]:
markets_comb_df.groupby(['MARKET_REG'])['Unit.slug'].nunique().to_frame()

In [ ]:
units_inspected_bycust = comb_df.groupby(['Customer.name'])['Unit.slug'].nunique().to_frame().reset_index()

In [ ]:
units_inspected_bycust['Unit.slug'].sum()

In [ ]:
## NOTE: in 2022 there were ZERO power inspections in February

In [ ]:
fig=plt.figure()
fig.show()
ax=fig.add_subplot(111)

ax.set_title('Gecko Power Inspections')
ax.set_ylabel('Count')
ax.set_xlabel('Month')
ax.set_xticks([1,3,4,5,6,7,8,9,10,11,12])
ax.set_xticklabels(gecko_2022_inspections['Inspection.month'])


ax.plot(gecko_2022_inspections['Inspection.month'], gecko_2022_inspections['Inspection.slug'], c='r', label='Gecko Power Inspections')


plt.legend()
plt.draw()

In [6]:
with open(r'../data/raw_data/gecko_tank_shells.json', encoding="utf8") as json_data:
    d = json.loads(json_data.read())
    json_data.close()

In [7]:
tank_shells_df = pd.json_normalize(d)

In [18]:
tank_shells_df.head()

,Component.tube_diameter,Component.material_type,Component.contents,Component.tube_spacing,Component.wall_nominal_thickness,Component.temperature,Component.sea_level,Component.wall_warning_thickness,Component.difficulty_factor_percentage,Component.id,Component.shell_diameter,Component.coating_nominal_thickness,Component.salesforce_id,Component.name,Component.slope_angle,Component.coating_warning_thickness,Component.description,Component.lat,Component.component_type,Component.slug,Component.lng,Component.sales_asset_type_id,Component.unit_id,Component.alloy_id,Unit.name,Unit.location_id,Unit.slug,Unit.capacity,Unit.inspection_code,Unit.latitude,Unit.retirement_date,Unit.total_tubes,Unit.description,Unit.id,Unit.built_date,Unit.fuel_type,Unit.inspection_interval_years,Unit.longitude,Unit.sales_asset_category_id,Unit.salesforce_id,Location.name,Location.address_line_2,Location.postal_code,Location.customer_id,Location.city,Location.slack_channel,Location.lat,Location.salesforce_id,Location.address_line_1,Location.id,Location.region,Location.country,Location.slug,Location.industry_type,Location.lng,Location.drive_folder_id,Customer.salesforce_id,Customer.logo,Customer.slug,Customer.id,Customer.name,Customer.drive_folder_id
0,0.0,carbon_steel,None,0.0,0.325,,NaN,0.15,0,18,NaN,0.0,,Outer Shell,NaN,0.0,The zero point is 0.5 feet from the left weld ...,None,2,outer-shell-a27f0e,None,NaN,18,NaN,Coal Silo 1,10,coal-silo-1-5ccad9,0,,0.0,None,0,,18,None,,0,0.0,NaN,,Warrior Run,,21502,10,Cumberland,insp_aes_warriorrun,39.59586,0011U00000EHUeuQAH,11600 Mexico Farms Rd SE,10,MD,US,warrior-run-134143,power,-78.74616,0B61E5R0McQ-1dzEwQ0JsMUdtVGs,0011U00000EHUZhQAP,customer_logos/aes.png,aes-ea4f30,10,AES,0B61E5R0McQ-1UmhZTThYRzJWNVU
1,0.0,carbon_steel,None,0.0,0.325,,NaN,0.15,0,19,NaN,0.0,,Outer Shell,NaN,0.0,The zero point is 5.33 feet from the right wel...,None,2,outer-shell-dfd932,None,NaN,19,NaN,Coal Silo 2,10,coal-silo-2-75fac9,0,,0.0,None,0,,19,None,,0,0.0,NaN,,Warrior Run,,21502,10,Cumberland,insp_aes_warriorrun,39.59586,0011U00000EHUeuQAH,11600 Mexico Farms Rd SE,10,MD,US,warrior-run-134143,power,-78.74616,0B61E5R0McQ-1dzEwQ0JsMUdtVGs,0011U00000EHUZhQAP,customer_logos/aes.png,aes-ea4f30,10,AES,0B61E5R0McQ-1UmhZTThYRzJWNVU
2,0.0,carbon_steel,None,0.0,0.325,,NaN,0.15,0,20,NaN,0.0,,Outer Shell,NaN,0.0,The zero point is 0.75 feet from left weld of ...,None,2,outer-shell-b5407c,None,NaN,20,NaN,Coal Silo 3,10,coal-silo-3-d2f1f1,0,,0.0,None,0,,20,None,,0,0.0,NaN,,Warrior Run,,21502,10,Cumberland,insp_aes_warriorrun,39.59586,0011U00000EHUeuQAH,11600 Mexico Farms Rd SE,10,MD,US,warrior-run-134143,power,-78.74616,0B61E5R0McQ-1dzEwQ0JsMUdtVGs,0011U00000EHUZhQAP,customer_logos/aes.png,aes-ea4f30,10,AES,0B61E5R0McQ-1UmhZTThYRzJWNVU
3,0.0,carbon_steel,None,0.0,0.375,,NaN,0.15,0,21,NaN,0.0,,Outer Shell,NaN,0.0,The zero point is 7.75 feet from the left weld...,None,2,outer-shell-dc6fbd,None,NaN,21,NaN,Coal Silo 4,10,coal-silo-4-fe90e8,0,,0.0,None,0,,21,None,,0,0.0,NaN,,Warrior Run,,21502,10,Cumberland,insp_aes_warriorrun,39.59586,0011U00000EHUeuQAH,11600 Mexico Farms Rd SE,10,MD,US,warrior-run-134143,power,-78.74616,0B61E5R0McQ-1dzEwQ0JsMUdtVGs,0011U00000EHUZhQAP,customer_logos/aes.png,aes-ea4f30,10,AES,0B61E5R0McQ-1UmhZTThYRzJWNVU
4,1.0,carbon_steel,None,2.0,NaN,,NaN,NaN,0,32,NaN,NaN,,Shell,NaN,NaN,,None,2,shell-48ca11,None,NaN,31,NaN,Coal Bunker - Ultrasound,12,coal-bunker-c04a12,0,,0.0,None,0,,31,None,,0,0.0,NaN,,Pittsburgh,,15208,12,Pittsburgh,,40.45465,None,6901 Lynn Way,12,PA,US,pittsburgh-9e6b4e,None,-79.90496,None,None,customer_logos/gecko.jpeg,demo-company-d5b267,12,Gecko Robotics (Legacy),None


In [10]:
tank_shells_df.columns

Index(['Component.tube_diameter', 'Component.material_type',
       'Component.contents', 'Component.tube_spacing',
       'Component.wall_nominal_thickness', 'Component.temperature',
       'Component.sea_level', 'Component.wall_warning_thickness',
       'Component.difficulty_factor_percentage', 'Component.id',
       'Component.shell_diameter', 'Component.coating_nominal_thickness',
       'Component.salesforce_id', 'Component.name', 'Component.slope_angle',
       'Component.coating_warning_thickness', 'Component.description',
       'Component.lat', 'Component.component_type', 'Component.slug',
       'Component.lng', 'Component.sales_asset_type_id', 'Component.unit_id',
       'Component.alloy_id', 'Unit.name', 'Unit.location_id', 'Unit.slug',
       'Unit.capacity', 'Unit.inspection_code', 'Unit.latitude',
       'Unit.retirement_date', 'Unit.total_tubes', 'Unit.description',
       'Unit.id', 'Unit.built_date', 'Unit.fuel_type',
       'Unit.inspection_interval_years', 'Unit.lo

In [9]:
tank_shells_df.shape

(1648, 62)

In [32]:
def is_liquor(string):
    truth = False
    if 'liq' in string.lower():
        truth = True
        
    return truth

In [33]:
test = 'liquor'

is_liquor(test)

True

In [34]:
tank_shells_df[tank_shells_df['Unit.name'].apply(lambda x: is_liquor(x)) | tank_shells_df['Unit.slug'].apply(lambda x: is_liquor(x))].shape

(14, 62)

In [35]:
liq_tanks_df = tank_shells_df[tank_shells_df['Unit.name'].apply(lambda x: is_liquor(x)) | tank_shells_df['Unit.slug'].apply(lambda x: is_liquor(x))]

In [36]:
liq_tanks_df

,Component.tube_diameter,Component.material_type,Component.contents,Component.tube_spacing,Component.wall_nominal_thickness,Component.temperature,Component.sea_level,Component.wall_warning_thickness,Component.difficulty_factor_percentage,Component.id,Component.shell_diameter,Component.coating_nominal_thickness,Component.salesforce_id,Component.name,Component.slope_angle,Component.coating_warning_thickness,Component.description,Component.lat,Component.component_type,Component.slug,Component.lng,Component.sales_asset_type_id,Component.unit_id,Component.alloy_id,Unit.name,Unit.location_id,Unit.slug,Unit.capacity,Unit.inspection_code,Unit.latitude,Unit.retirement_date,Unit.total_tubes,Unit.description,Unit.id,Unit.built_date,Unit.fuel_type,Unit.inspection_interval_years,Unit.longitude,Unit.sales_asset_category_id,Unit.salesforce_id,Location.name,Location.address_line_2,Location.postal_code,Location.customer_id,Location.city,Location.slack_channel,Location.lat,Location.salesforce_id,Location.address_line_1,Location.id,Location.region,Location.country,Location.slug,Location.industry_type,Location.lng,Location.drive_folder_id,Customer.salesforce_id,Customer.logo,Customer.slug,Customer.id,Customer.name,Customer.drive_folder_id
288,0.0,carbon_steel,None,0.0,0.375,,NaN,0.22,0,722,8.0,0.0000,,Stack Wall,NaN,0.0,,None,2,tank-wall-f8e2fb,None,NaN,358,NaN,Black Liquor Stack,112,black-liquor-stack-2e7567,0,,0.0,None,0,,358,None,,0,0.0,NaN,,Nekoosa Pulp and Paper Mill,,54457,79,Nekoosa,insp_domtar_nekoosa,44.313530,0011U00000TFUW8QAP,301 Point Basse Avenue,112,WI,US,nekoosa-pulp-and-paper-mill-226e84,paper,-89.895990,16K6-Z9KqfUOUfuBu7eT4KmHScsrS2pJU,0011U00000EHUWdQAP,customer_logos/domtar.png,domtar-745a5a,79,Domtar,1OFk4M3IIZyX1uNkRKADw_rSFfuBqMLNP
717,0.0,carbon_steel,None,0.0,0.550,,NaN,0.40,0,1570,0.9,NaN,,Line,NaN,NaN,,None,2,line-4cfca8,None,NaN,734,NaN,White Liquor Line,101,white-liquor-line-46e33e,0,,0.0,None,0,,734,None,,0,0.0,NaN,,Brewton Mill,,36427,8,Brewton,insp_gp_brewton,31.079360,0011U00000n94wrQAA,32224 Highway 31,101,AL,US,brewton-mill-fb51be,paper,-87.115270,1f74rBSA-STUvhgbgGwLAVrVIRPGPcXwQ,0011U00000EHUN6QAP,customer_logos/georgia-pacific-300x225.jpg,georgia-pacific-a1ec53,8,Georgia Pacific,1mHDa3E68Ydr1gNpfmSttRQTolkXQkgaM
1254,0.0,carbon_steel,,0.0,0.000,ambient,NaN,0.00,0,3061,35.0,0.3125,a0M1U00000Jew3LUAR,Tank Shell,NaN,NaN,,None,2,tank-shell-5fe88d,None,9.0,1518,1.0,Weak Black Liquor Storage Tank,140,weak-black-liquor-storage-tank-00747b,0,653,0.0,None,0,,1518,None,,0,0.0,5.0,02i1U000004RshDQAS,Ashdown,,71822,79,Ashdown,insp_domtar_ashdown,33.642370,0011U00000EHUZOQA5,285 US-71,140,AR,US,ashdown-a02e36,paper,-94.100980,1nsCEtWfTBzmBbDJ11sPSGus5GU_cvyxh,0011U00000EHUWdQAP,customer_logos/domtar.png,domtar-745a5a,79,Domtar,1OFk4M3IIZyX1uNkRKADw_rSFfuBqMLNP
1255,0.0,carbon_steel,,0.0,0.250,ambient,NaN,0.20,0,3063,10.0,NaN,None,Tank Shell,NaN,NaN,,None,2,tank-shell-df38c4,None,NaN,1520,NaN,Black Liquor Flash Tank,437,black-liquor-flash-tank-8398a2,0,None,0.0,None,0,,1520,None,None,0,0.0,NaN,None,Monticello Kraft Pulp & Linerboard Mill,,39654-7601,8,Monticello,insp_gp_monticello_cs,31.623700,0011U00000PTKntQAH,604 NA Sandifer Road,437,,US,georgia-pacific-monticello-kraft-pulp-liner-d1...,paper,-90.082740,12ZrTGOxiFC-sVU2VXwTVadxGLFT6LRkI,0011U00000EHUN6QAP,customer_logos/georgia-pacific-300x225.jpg,georgia-pacific-a1ec53,8,Georgia Pacific,1mHDa3E68Ydr1gNpfmSttRQTolkXQkgaM
1256,0.0,carbon_steel,,0.0,0.250,ambient,NaN,0.20,0,3064,10.0,NaN,None,Tank Shell,NaN,NaN,,None,2,tank-shell-154beb,None,NaN,1520,NaN,Black Liquor Flash Tank,437,black-liquor-flash-tank-8398a2,0,None,0.0,None,0,,1520,None,None,0,0.0,NaN,None,Monticello Kraft Pulp & Linerboard Mill,,39654-7601,8,Monticello,insp_gp_monticello_cs,31.623700,0011U00000PTKntQAH,604 NA Sandifer Road,437,,US,georgia-pacific-monticello-kraft-pulp-liner-d1...,paper,-90.082740,12ZrTGOxiFC-sVU2VXwTVadxGLFT6LRkI,0011U00000EHUN6QAP,customer_logos/georgia-pacific-300x225.jpg,georgia-

In [37]:
liq_tanks_df[tank_shells_df['Customer.id']==8].shape

C:\Users\tom.fiorelli\AppData\Local\Temp\ipykernel_14968\2539260755.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  liq_tanks_df[tank_shells_df['Customer.id']==8].shape


(6, 62)

In [38]:
liq_tanks_df[tank_shells_df['Customer.id']==8]

C:\Users\tom.fiorelli\AppData\Local\Temp\ipykernel_14968\1999285972.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  liq_tanks_df[tank_shells_df['Customer.id']==8]


,Component.tube_diameter,Component.material_type,Component.contents,Component.tube_spacing,Component.wall_nominal_thickness,Component.temperature,Component.sea_level,Component.wall_warning_thickness,Component.difficulty_factor_percentage,Component.id,Component.shell_diameter,Component.coating_nominal_thickness,Component.salesforce_id,Component.name,Component.slope_angle,Component.coating_warning_thickness,Component.description,Component.lat,Component.component_type,Component.slug,Component.lng,Component.sales_asset_type_id,Component.unit_id,Component.alloy_id,Unit.name,Unit.location_id,Unit.slug,Unit.capacity,Unit.inspection_code,Unit.latitude,Unit.retirement_date,Unit.total_tubes,Unit.description,Unit.id,Unit.built_date,Unit.fuel_type,Unit.inspection_interval_years,Unit.longitude,Unit.sales_asset_category_id,Unit.salesforce_id,Location.name,Location.address_line_2,Location.postal_code,Location.customer_id,Location.city,Location.slack_channel,Location.lat,Location.salesforce_id,Location.address_line_1,Location.id,Location.region,Location.country,Location.slug,Location.industry_type,Location.lng,Location.drive_folder_id,Customer.salesforce_id,Customer.logo,Customer.slug,Customer.id,Customer.name,Customer.drive_folder_id
717,0.0,carbon_steel,None,0.0,0.55,,NaN,0.4,0,1570,0.9,NaN,,Line,NaN,NaN,,None,2,line-4cfca8,None,NaN,734,NaN,White Liquor Line,101,white-liquor-line-46e33e,0,,0.0,None,0,,734,None,,0,0.0,NaN,,Brewton Mill,,36427,8,Brewton,insp_gp_brewton,31.07936,0011U00000n94wrQAA,32224 Highway 31,101,AL,US,brewton-mill-fb51be,paper,-87.11527,1f74rBSA-STUvhgbgGwLAVrVIRPGPcXwQ,0011U00000EHUN6QAP,customer_logos/georgia-pacific-300x225.jpg,georgia-pacific-a1ec53,8,Georgia Pacific,1mHDa3E68Ydr1gNpfmSttRQTolkXQkgaM
1255,0.0,carbon_steel,,0.0,0.25,ambient,NaN,0.2,0,3063,10.0,NaN,None,Tank Shell,NaN,NaN,,None,2,tank-shell-df38c4,None,NaN,1520,NaN,Black Liquor Flash Tank,437,black-liquor-flash-tank-8398a2,0,None,0.0,None,0,,1520,None,None,0,0.0,NaN,None,Monticello Kraft Pulp & Linerboard Mill,,39654-7601,8,Monticello,insp_gp_monticello_cs,31.62370,0011U00000PTKntQAH,604 NA Sandifer Road,437,,US,georgia-pacific-monticello-kraft-pulp-liner-d1...,paper,-90.08274,12ZrTGOxiFC-sVU2VXwTVadxGLFT6LRkI,0011U00000EHUN6QAP,customer_logos/georgia-pacific-300x225.jpg,georgia-pacific-a1ec53,8,Georgia Pacific,1mHDa3E68Ydr1gNpfmSttRQTolkXQkgaM
1256,0.0,carbon_steel,,0.0,0.25,ambient,NaN,0.2,0,3064,10.0,NaN,None,Tank Shell,NaN,NaN,,None,2,tank-shell-154beb,None,NaN,1520,NaN,Black Liquor Flash Tank,437,black-liquor-flash-tank-8398a2,0,None,0.0,None,0,,1520,None,None,0,0.0,NaN,None,Monticello Kraft Pulp & Linerboard Mill,,39654-7601,8,Monticello,insp_gp_monticello_cs,31.62370,0011U00000PTKntQAH,604 NA Sandifer Road,437,,US,georgia-pacific-monticello-kraft-pulp-liner-d1...,paper,-90.08274,12ZrTGOxiFC-sVU2VXwTVadxGLFT6LRkI,0011U00000EHUN6QAP,customer_logos/georgia-pacific-300x225.jpg,georgia-pacific-a1ec53,8,Georgia Pacific,1mHDa3E68Ydr1gNpfmSttRQTolkXQkgaM
1257,0.0,carbon_steel,,0.0,0.25,ambient,NaN,0.2,0,3065,30.0,NaN,None,Tank Shell,NaN,NaN,,None,2,tank-shell-0384de,None,NaN,1520,NaN,Black Liquor Flash Tank,437,black-liquor-flash-tank-8398a2,0,None,0.0,None,0,,1520,None,None,0,0.0,NaN,None,Monticello Kraft Pulp & Linerboard Mill,,39654-7601,8,Monticello,insp_gp_monticello_cs,31.62370,0011U00000PTKntQAH,604 NA Sandifer Road,437,,US,georgia-pacific-monticello-kraft-pulp-liner-d1...,paper,-90.08274,12ZrTGOxiFC-sVU2VXwTVadxGLFT6LRkI,0011U00000EHUN6QAP,customer_logos/georgia-pacific-300x225.jpg,georgia-pacific-a1ec53,8,Georgia Pacific,1mHDa3E68Ydr1gNpfmSttRQTolkXQkgaM
1317,0.0,carbon_steel,,0.0,0.50,ambient,NaN,0.1,0,3246,45.0,NaN,None,Tank Shell,NaN,NaN,,None,2,tank-shell-a93e15,None,NaN,1626,NaN,#4 Weak Black Liquor Tank,101,4-weak-black-liquor-tank-706057,0,None,0.0,None,0,,1626,None,None,0,0.0,NaN,None,Brewton Mill,,36427,8,Brewton,insp_gp_brewton,31.07936,0011U00000n94wrQAA,32224 Highway 31,101,AL,US,brewton-mill-fb51be,paper,-87.11527,1f74rBSA-STUvhgbg

1648 tank shell inspections, 14 of those are liquor tanks, 6 are at GP